In [ ]:
import numpy as np
import pandas as pd

!pip install stanford-corenlp
!pip install stanfordnlp
!pip install pytorch-transformers
!pip install transformers

import random
import pickle
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import os,sys,time
import corenlp
import stanfordnlp
from transformers import RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer, BertModel, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW

sys.path.append('./scripts/')
stanfordnlp.download('en')

from event_dataset import EventReader, SentenceReader, Parser
from word_models import EventExtractor
from da_models_new import AdversarialEventExtractor, GradReverse
from bert_embedding_extractor import BertFeatureExtractor
import util_sentence_classifier as util_sent_cf

In [ ]:
args = {}

args["data_dir"] = "./data"
args["train_file"] = "train.ids"
args["dev_file"] = "dev.ids"
args["test_file"] = "test.ids"
args["emb_file"] = "./data/guten.vectors.txt"

args["batch_size"] = 16
args["emb_size"] = 100
args["hidden_size"] = 100
args["dropout"] = 0.5
args["num_epochs"] = 100
args["learning_rate"] = 0.001
args["bidir"] = True
args["seed"] = 0
args["do_train"] = True
args["do_eval"] = True
args["model"] = "word"
args["save_path"] = None
args["suffix"] = None
args["num_layers"] = 3
args["oov_vocab"] = None

random.seed(args["seed"])
torch.manual_seed(args["seed"])
np.random.seed(args["seed"])

use_cuda = torch.cuda.is_available()

reader = EventReader()
parser = Parser()

train_sentences, train_events = reader.read_events(args["data_dir"],args["train_file"])
# dev_sentences, dev_events = reader.read_events(args["data_dir"], args["dev_file"])
dev_sentences, dev_events = reader.read_events(args["data_dir"], args["test_file"])
test_sentences, test_events = reader.read_events(args["data_dir"], args["test_file"])

train_parse = parser.parse_sequences(train_sentences)
dev_parse = parser.parse_sequences(dev_sentences)
test_parse = parser.parse_sequences(test_sentences)

train_ = util_sent_cf.get_sentences_events(train_sentences,train_events)
dev_ = util_sent_cf.get_sentences_events(dev_sentences,dev_events)
test_ = util_sent_cf.get_sentences_events(test_sentences,test_events)

train_dataloader = util_sent_cf.tokenize_sentences_make_dataloader(train_,2)
dev_dataloader = util_sent_cf.tokenize_sentences_make_dataloader(dev_,2)
test_dataloader = util_sent_cf.tokenize_sentences_make_dataloader(test_,2)

In [ ]:
def sentence_event_classifier(model,sentence_in):

    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    sentences = tokenizer(sentence_in,padding = True,truncation=True,max_length = 512,return_tensors='pt')
    output = model(sentences.input_ids,attention_mask=sentences.attention_mask)
    logit = output[0].detach().cpu().numpy()
    pred = np.argmax(logit,axis=1).flatten()

    return pred

model_sent = RobertaForSequenceClassification.from_pretrained("roberta-base",num_labels = 2)
checkpoint = torch.load("./trained_model/model_sent_3_0.8705583756345178")
model_sent.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
def train(sentence_prediction, model_sent, dev_sentences, model, train_batches, dev_batches, num_epochs, learning_rate, use_cuda, path, model_type):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(list(model.rep_learner.parameters()) + list(model.classifier.parameters()), lr=learning_rate)
    best_precision, best_recall, best_f1 = 0.0, 0.0, 0.0
    for epoch in range(num_epochs):
        total_loss = 0.0
        num_batches = len(train_batches)
        for batch in train_batches:
            optimizer.zero_grad()
            batch = [x.to('cuda') for x in batch]
            labels, predictions = batch[-3], []
            labels = labels.contiguous().view(-1,1)
            if model_type == "word" or model_type == "delex" or model_type.startswith("bert"):
                sents, _, lengths, masks = batch
                predictions = model(sents, lengths.cpu(), masks)
            elif model_type == "pos":
                sents, pos, _, lengths, masks = batch
                predictions = model(sents, lengths.cpu(), masks, pos)
            loss = criterion(predictions, labels)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        total_loss /= num_batches
        print("Training Loss at epoch {}: {}".format(epoch, total_loss))
        print("Performance on development set:")
        precision, recall, f1 = test(sentence_prediction, model_sent, dev_sentences, model, dev_batches, use_cuda, '', args["model"])
        if f1 > best_f1:
            best_precision = precision
            best_recall = recall
            best_f1 = f1
            torch.save(model.state_dict(), path)
        model.train()

def test(sentence_prediction, model_sent, dev_sentences, model, dev_batches, use_cuda, path, model_type, oov=None):
    if path != '':
        model.load_state_dict(torch.load(path))
    model.eval()
    index = 0
    predicted, gold, correct = 0.0, 0.0, 0.0
    # all_reps = []
    iv_predicted, iv_gold, iv_correct = 0.0, 0.0, 0.0
    oov_predicted, oov_gold, oov_correct = 0.0, 0.0, 0.0
    for batch in dev_batches:
        cpu_sents = batch[0].view(-1,1)
        batch = [x.to('cuda') for x in batch]
        labels, predictions = batch[-3], []
        labels = labels.contiguous().view(-1,1)
        
        # Integrating my code here
        sent_predictions = []
        for j in range(len(batch[2])):
            length = batch[2][j]
            for i in range(length):
                sent_predictions.append(sentence_prediction[index+j])
                
        if model_type == "word" or model_type == "delex" or model_type.startswith("bert"):
            sents, _, lengths, masks = batch
            predictions = model(sents, lengths.cpu(), masks)   # Remove reps after dumping BERT
        elif model_type == "pos":
            sents, pos, _, lengths, masks = batch
            predictions = model(sents, lengths.cpu(), masks, pos)
        if use_cuda:
            predictions = predictions.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            # reps = reps.cpu()  # Comment out after dumping BERT
        cur_correct, cur_pred, cur_gold = calculate_batch_f1(predictions.tolist(), labels.tolist(), sent_predictions)
        if oov is not None:
            cur_iv_correct, cur_iv_pred, cur_iv_gold, cur_oov_correct, cur_oov_pred, cur_oov_gold = calculate_split_f1(predictions.tolist(), labels.tolist(), oov, cpu_sents, sent_vocab)
            iv_correct += cur_iv_correct
            iv_gold += cur_iv_gold
            iv_predicted += cur_iv_pred
            oov_correct += cur_oov_correct
            oov_gold += cur_oov_gold
            oov_predicted += cur_oov_pred
        predicted += cur_pred
        gold += cur_gold
        correct += cur_correct
        index += len(batch[0])
        # all_reps.append(reps)
    # pickle.dump(all_reps, open('BERT_reps_rec.pkl', 'wb'))
    # print('Dumped BERT record reps')
    precision = correct / predicted if predicted != 0 else 0.0
    recall = correct / gold if gold != 0 else 0.0
    f1 = (2 * precision * recall) / (precision + recall) if precision + recall != 0 else 0.0
    print("Precision: {}".format(precision))
    print("Recall: {}".format(recall))
    print("F1 Score: {}".format(f1))
    return precision, recall, f1

def calculate_split_f1(preds, labels, oov, raw_sents, sent_vocab):
    iv_predicted, iv_gold, iv_correct = 0.0, 0.0, 0.0
    oov_predicted, oov_gold, oov_correct = 0.0, 0.0, 0.0
    raw_sents = raw_sents.tolist()
    # print(len(raw_sents))
    # exit(1)
    i = 0
    for pred, label in zip(preds, labels):
        pred = 0 if pred[0] <= 0.0 else 1
        label = label[0]
        cur_words = raw_sents[i]
        #if pred == 1:
        i += 1

def calculate_batch_f1(preds, labels, sent_preds):
    predicted = 0.0
    gold = 0.0
    correct = 0.0
    for pred, label, sent_pred in zip(preds, labels, sent_preds):
        pred = (0*sent_pred) if pred[0] <= 0.0 else (1*sent_pred)
        label = label[0]
        if pred == 1:
            predicted += 1
        if label == 1:
            gold += 1
        if pred == label and label == 1:
            correct += 1
    return correct, predicted, gold

In [ ]:
'''
ONLY take the sentences which have atleast 1 label
'''

def filter_sentences(sentences,events,dataframe):
    filtered_sentences = []
    filtered_events = []
    for i in list(dataframe[dataframe.label==1].index):
        filtered_sentences.append(sentences[i])
        filtered_events.append(events[i])
        
    return filtered_sentences,filtered_events

f_train_sentences,f_train_events = filter_sentences(train_sentences,train_events,train_)
f_dev_sentences,f_dev_events = filter_sentences(dev_sentences,dev_events,dev_)
f_test_sentences,f_test_events = filter_sentences(dev_sentences,dev_events,dev_)

In [ ]:
sent_vocab = reader.construct_vocab(train_sentences + dev_sentences + test_sentences)
pos_vocab = reader.construct_vocab(train_parse + dev_parse + test_parse)
label_vocab = {"O": 0, "EVENT": 1, "ENT": 0}

use_shared_vocab = True
use_filtered_sentences_only = False

args["model_path"] = "./"
oov_vocab = None
if args["oov_vocab"] is not None:
    pickle.load(open(args["oov_vocab"], 'rb'))

if args["do_train"]:
    pickle.dump(pos_vocab, open(args["model_path"]+"_posvocab_{}.pkl".format(args["seed"]), "wb"))
    if use_shared_vocab:
        sent_vocab = pickle.load(open("./scripts/shared_vocab_news_lit.pkl", "rb"))
    pickle.dump(sent_vocab, open(args["model_path"]+"_vocab_{}.pkl".format(args["seed"]), "wb"))
elif args["do_eval"]:
    pos_vocab = pickle.load(open(args["model_path"]+"_posvocab_{}.pkl".format(args["seed"]), "rb"))
    sent_vocab = pickle.load(open(args["model_path"]+"_vocab_{}.pkl".format(args["seed"]), "rb"))

if(use_filtered_sentences_only):
    int_train_sents = reader.construct_integer_sequences(f_train_sentences, sent_vocab)
    int_train_labels = reader.construct_integer_sequences(f_train_events, label_vocab)
    int_dev_sents = reader.construct_integer_sequences(dev_sentences, sent_vocab)
    int_dev_labels = reader.construct_integer_sequences(dev_events, label_vocab)
    int_test_sents = reader.construct_integer_sequences(test_sentences, sent_vocab) # made changes !!!!!!!!!!!! LOOK AT IT
    int_test_labels = reader.construct_integer_sequences(test_events, label_vocab)
else:
    int_train_sents = reader.construct_integer_sequences(train_sentences, sent_vocab)
    int_train_labels = reader.construct_integer_sequences(train_events, label_vocab)
    int_dev_sents = reader.construct_integer_sequences(dev_sentences, sent_vocab)
    int_dev_labels = reader.construct_integer_sequences(dev_events, label_vocab)
    int_test_sents = reader.construct_integer_sequences(test_sentences, sent_vocab)
    int_test_labels = reader.construct_integer_sequences(test_events, label_vocab)

train_batches, dev_batches, test_batches = [], [], []

if args["model"] == "word":
    train_batches = reader.create_padded_batches(int_train_sents, int_train_labels, args["batch_size"], use_cuda, True)
    dev_batches = reader.create_padded_batches(int_dev_sents, int_dev_labels, args["batch_size"], use_cuda, False)
    test_batches = reader.create_padded_batches(int_test_sents, int_test_labels, args["batch_size"], use_cuda, False)

In [ ]:
if args["model"] == 'word':
    model = EventExtractor(len(list(sent_vocab.keys())), args["emb_size"], args["hidden_size"], 1, args["dropout"], args["bidir"], args["model"])

if args["emb_file"] is not None:
    model.rep_learner.load_embeddings(args["emb_file"], sent_vocab)
if use_cuda:
    model = model.cuda()
if args["do_train"]:
    sentence_prediction = []
    for index in range(0,len(dev_sentences),16):
        for s in sentence_event_classifier(model_sent, [" ".join(dev_sentences[x]) for x in range(index,min(index+16,len(dev_sentences)))] ):
            sentence_prediction.append(1)
    train(sentence_prediction, model_sent, dev_sentences, model, train_batches, dev_batches, args["num_epochs"], args["learning_rate"], use_cuda, args["model_path"]+"_{}.pth".format(args["seed"]), args["model"])
    if args["do_eval"]:
        test(sentence_prediction, model_sent, dev_sentences, model, test_batches, use_cuda, args["model_path"]+"_{}.pth".format(args["seed"]), args["model"], oov=oov_vocab)
else:
    sentence_prediction = []
    for index in range(0,len(dev_sentences),16):
        for s in sentence_event_classifier(model_sent, [" ".join(dev_sentences[x]) for x in range(index,min(index+16,len(dev_sentences)))] ):
            sentence_prediction.append(1)
    test(sentence_prediction, model_sent, dev_sentences, model, test_batches, use_cuda, args.model_path+"_{}.pth".format(args.seed), args.model, oov=oov_vocab)